**Steps involved :**

**Data Collection**
* Preprocessing the Data
* Splitting the data(spectrum) into separate filter lengths
* Passing the filters through a CNN for reconstruction of the spectrum
* We can construct spectrum from just an image
---
**For the redshift part :**

* Soften the data using Savitzky Golay filter
* Construct the XGBoost model from scratch or from library
* Predict the redshift
* Compare the results

**Step 1 : Data collection :**
The cnn model we are using almost require 700 datasets. Which include 4 filters of the galaxy and it's spectrum respectively.But Downloading such mass of a dataset manually isn't possible and we couldn't find a ready set online. SO we had to create an automation software to mimic the manual entry and download.

* For the automation task we employed a library named Selenium. And the site from which the data is being downloaded is SDSS sky survey. The Automation process includes, Entering into the website and navigating to find the form regarding the SpaceID of the object of interest. The program then fills the form from one of the ID's stored in a CSV file tailored by us selecting suitable galaxies.
* After navigating, a new page is loaded to download the filters of the galaxy in a corrected frame.


---


* The file is then downloaded and stored in the following format:
* Data/galaxy/galaxy_id_filter.fits.bz2/galaxy_id_spectrum.fits.bz2

---

* Then the files from which in format of fits.bz2 is unpacked to .fits format.

In [1]:
# Importing the base modules
import pandas as pd
import numpy as np
import os
import shutil
import glob
import time
import bz2

# Import the web automation library
from selenium import webdriver
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options

# Intitializing the options to an empty function to prevent errors
options = Options()

# Initializing the chrome driver


# Enter the SDSS data collection webpage
class Id_collection:
    def __init__(self,n_points,csv_file):
        self.n_points = n_points
        self.csv_file = csv_file
    def get_Ids(self):
        file = pd.read_csv(self.csv_file)
        ids = file["specobj_id"].head(self.n_points).tolist()
        SpaceIds = []
        for i in range(len(ids)):
            SpaceIds.append(int(ids[i][1:-1]))
        return SpaceIds
    def url_const(self,SpaceIds):
        urls = []
        for i in range(len(SpaceIds)):
            urls.append(f"http://cas.sdss.org/dr18/VisualTools/explore/summary?sId={SpaceIds[i]}")
        return urls
    def create_folder(SpaceId):
        galaxy_folder = f"Galaxy_{SpaceId}"
        images_folder = os.path.join(galaxy_folder,"Images")
        os.makedirs(images_folder,exist_ok=True)
        return images_folder
    def move_rename(download_dir,images_folder,filter_name):

        down_file_latest = max(glob.glob(os.path.join(download_dir,"*")),key = os.path.getctime)
            # Unpack the .bz2 file to .fits file
        with bz2.open(down_file_latest, 'rb') as bz2_file:
            with open(down_file_latest[:-4], 'wb') as fits_file:  # Remove the .bz2 extension
                fits_file.write(bz2_file.read())
        new_file_name = os.path.join(images_folder,f"{filter_name}_filter.fits")
        shutil.move(down_file_latest,new_file_name)

    
# Intitialize the Id collection class
Id_collection = Id_collection(10,"./optical_search_412544.csv")
spaceIds = Id_collection.get_Ids()
#print(spaceIds)
url_collection = Id_collection.url_const(spaceIds)
#print(url_collection)

# Now we have the urls for all n_points number of galxies in a list
# Next part is to access the url and download the filters and spectrum

def navigate_to_fits(url):
    driver = webdriver.Firefox()
    driver.get(url)
    # Define an explicit wait time (e.g., 15 seconds)
    wait = WebDriverWait(driver, 15)

    # Wait until the "FITS" link becomes clickable
    fits_link = wait.until(EC.element_to_be_clickable((By.PARTIAL_LINK_TEXT, 'FITS')))

    # Click the "FITS" link once it's clickable
    fits_link.click()
    
    # Switchcing the tab
    driver.switch_to.window(driver.window_handles[1])

    # Download all filters in corrected frames
    filters = ["u","g","r","i","z"]

    # Loop for all the filters and download the first link.
    for filter in filters:
        wait_time = WebDriverWait(driver,10)
        download_link = wait_time.until(EC.presence_of_element_located((By.LINK_TEXT,filter)))
        time.sleep(1)
        download_link.click()
    time.sleep(2)
    driver.quit()
    
    

navigate_to_fits(url_collection[0])





    


UnexpectedAlertPresentException: Alert Text: None
Message: Dismissed user prompt dialog: If you exit now, 1 download will be canceled. Are you sure you want to exit?
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
UnexpectedAlertOpenError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:800:5
GeckoDriver.prototype._handleUserPrompts@chrome://remote/content/marionette/driver.sys.mjs:2846:13
